In [1]:
# Thsi is the code used for the paper results
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import os
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct
import matplotlib.pyplot as plt


In [2]:
params = {
    # exp
    "seed": 10,
    "runs": 100,
    "exp_name": ct.generate_readable_short_id("synthetic_mg"),
    "path": "../../",
    "split": "random_split", #CV, random_split

    "test_split": 0.0588,
    "calib_split": 0.5,
    
    "plot": True,
    
    "calib_methods": [
                      "RF_d",
                      "RF_opt", 
                      "RF_large", 
                      "Platt", "ISO", "Beta", "VA",
                      "CT", "PPA",
                      "Rank"
                      ],

    "calib_method_colors": {
        "RF_d": "blue", 
        "RF_opt": "orange", 
        "RF_large": "red",
        "Platt": "Brown", 
        "ISO": "purple", 
        "Beta": "magenta", 
        "VA": "gray",
        "CT": "slategray",
        "PPA": "olive", 
        "Rank": "silver"     
    },

    
    "metrics": ["acc", "brier", "ece", "logloss", "tce_kl", "tce_mse"], 

    #data
    "data_name": "synthetic_mg",
    "plot_data": True,
    "data_size": 1700,
    "n_features": 2,

    "bais_accuracy": 0,
    
    "class1_mean_min":0, 
    "class1_mean_max":1,
    "class2_mean_min":1, 
    "class2_mean_max":3, 

    "class1_cov_min":1, 
    "class1_cov_max":2,
    "class2_cov_min":1, 
    "class2_cov_max":2, 

    # "class1_mean_min":0, 
    # "class1_mean_max":1,
    # "class2_mean_min":1, 
    # "class2_mean_max":3, 

    # "class1_cov_min":4, 
    # "class1_cov_max":5,
    # "class2_cov_min":1, 
    # "class2_cov_max":2, 


    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":50,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [10],
                    "max_depth": np.arange(2, 100).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf":  np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False],
                    },
    
    "oob": False,
    "laplace": 0, # if set to 1, OOB must be false
    "curt_v": np.arange(1, 100).tolist(),

    "exp_key": "calib_size",
    "exp_values":  np.arange(2, 100, 5).tolist()
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param 2 done
exp_param 7 done
exp_param 12 done
exp_param 17 done
exp_param 22 done
exp_param 27 done
exp_param 32 done
exp_param 37 done
exp_param 42 done
exp_param 47 done
exp_param 52 done
exp_param 57 done
exp_param 62 done
exp_param 67 done
exp_param 72 done
exp_param 77 done
exp_param 82 done
exp_param 87 done
exp_param 92 done
exp_param 97 done


In [4]:

plot_calib_methods = params["calib_methods"].copy()
# plot_calib_methods.remove("RF_d")
# plot_calib_methods.remove("RF_opt")
# plot_calib_methods.remove("RF_large")
# plot_calib_methods.remove("Platt")
# plot_calib_methods.remove("Beta")
# plot_calib_methods.remove("CT")
# plot_calib_methods.remove("PPA")
plot_calib_methods

['RF_d',
 'RF_opt',
 'RF_large',
 'Platt',
 'ISO',
 'Beta',
 'VA',
 'CT',
 'PPA',
 'Rank']

In [6]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=False, 
                                    std=True)

path = f"results/{params['exp_name']}/calib_size_zoom"
if not os.path.exists(path):
    os.makedirs(path)

max_value = [None, None, 0.1, 2, 0.15, 0.015]
min_value = [None, None, None, None, 0.02, 0.006]

# max_value = [None, None, None, None, None, None]
# min_value = [None, None, None, None, None, None]


for metric, max_v, min_v in zip(params["metrics"], max_value, min_value): #
    ax = tables[metric][plot_calib_methods].plot(color=params["calib_method_colors"])
    if metric == "acc":
        metric_p = "ACC"
    elif metric == "logloss":
        metric_p = "LogLoss"
    elif metric == "ece":
        metric_p = "ECE"
    elif metric == "brier":
        metric_p = "Brier"
    elif metric == "tce_kl":
        metric_p = "TCE_KL"
    elif metric == "tce_mse":
        metric_p = "TCE_MSE"
    ax.set_xlabel("Calibration set size %")
    ax.set_ylabel(metric_p)
    plt.ylim(min_v, max_v)
    plt.savefig(f"{path}/cs_{metric}.pdf", format='pdf', transparent=True)
    plt.close() 
